# RAG using Llama-3

### Loading libraries and dependencies

In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

### Pointing the data path

In [3]:
input_dir_path = './data'

### Loading embedding and LLM model

Check more here:

- [Hugging Face Transformers](https://huggingface.co/models)
- [Ollama ](https://ollama.io/)

In [4]:
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Settings.llm = Ollama(model="llama3.2:1b", request_timeout=360.0)

### Loading the data

In [5]:
documents = SimpleDirectoryReader(
                input_dir=input_dir_path,
                required_exts=[".pdf"],
                recursive=True
            ).load_data()

### Data embedding

In [7]:
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine()

### Query and response

In [11]:
response = query_engine.query("Hàng hóa của tôi bị thất lạc thì bồi thường bao nhiêu?")
print(response)

3.000.000đ là mức bồi thường cho bất kỳ đơn hàng nào không được giao trên đường đi hoặc bị hư hỏng trong quá trình vận chuyển mà không có sự thỏa thuận giữa hai bên.


### Adding some custom prompt

In [20]:
qa_prompt_tmpl_str = ("""
Thông tin ngữ cảnh được cung cấp dưới đây.  
---------------------  
{context_str}  
---------------------  
Dựa vào thông tin ngữ cảnh ở trên, hãy phân tích kỹ lưỡng và trả lời câu hỏi một cách chi tiết, ngắn gọn, và rõ ràng.  
Câu hỏi: Vui lòng mô tả chính sách giao hàng của công ty, bao gồm:  
1. Thời gian giao hàng.  
2. Phí vận chuyển.  
3. Các khu vực được giao và khu vực bị giới hạn (nếu có).
Query: {query_str}
Câu trả lời:                      
"""
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

### Query and response

In [21]:
response = query_engine.query("Thời gian giao hàng là bao lâu?")
print(response)

Thông tin ngữ cảnh cung cấp về chính sách giao hàng của SuperShip, một công ty vận chuyển hàng hóa chuyên nghiệp. Dưới đây là phân tích chi tiết về các yếu tố liên quan đến thời gian giao hàng:

- **Thời gian lấy hàng**: Đây là 2 trường hợp cụ thể được đề cập trong câu trả lời.
  - Trường hợp 1: Khi yêu cầu lấy hàng thông qua điện thoại trước 10h30, đơn hàng sẽ được giao trong khoảng 10 giờ ngày hôm đó. 
  - Trường hợp 2: Khi yêu cầu lấy hàng thông qua điện thoại trước 16h00, đơn hàng cũng sẽ được giao vào lúc 4 giờ tối đó.

- **Phí vận chuyển**: Như đã đề cập trong phần 2.8 Chính sách “lưu kho”, phí lưu kho là chi phí để SuperShip hỗ trợ khách hàng lưu trữ sản phẩm miễn phí, trong trường hợp không vượt quá diện tích lưu kho 1m³ (trong trường hợp này có thể thay đổi tùy theo tình hình thực tế).

- **Các khu vực được giao và khu vực bị giới hạn**: Không có thông tin cụ thể về các khu vực được giao hay bị giới hạn. Tuy nhiên, dựa trên việc phân loại các trường hợp lấy hàng và lưu kho, ch